In [ ]:
import numpy as np
import pandas as pd

#seed
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import applications
#from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [ ]:
#applying transfer learning
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

In [ ]:
# Freeze first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

In [ ]:
# adding layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(62, activation="softmax")(x)

In [ ]:
model_final = Model(input = model.input, output = predictions)

In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 30,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    directory="img/train/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = test_datagen.flow_from_directory(
    directory="img/valid/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory="img/test/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [ ]:
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model_final.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30,
                    callbacks = [checkpoint, early]
)

In [ ]:
model_json = model_final.to_json()
with open("model_image.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model_final.save_weights("image_model_weights.h5")
print("Saved model to disk")

In [ ]:
model_final.save_weights("vgg16_1.h5")
print("Saved model to disk")

In [ ]:
json_file = open('model_image.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
# load weights into new model
loaded_model.load_weights("image_model_weights_2.h5")
print("Loaded model from disk")

In [ ]:
test_generator.reset()
pred=loaded_model.predict_generator(test_generator,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [ ]:
results[['directory', 'file']] = results.Filename.str.split("\\", expand = True)

In [ ]:
results.to_csv('test_keras.csv', index = False)

In [ ]:
(results.Predictions == results.directory).mean()

In [ ]:
df_label = pd.DataFrame(list(labels.values()), columns = ['class'])

In [ ]:
df_label.to_csv('labels.csv', index = False)

In [21]:
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import model_from_json
import pandas as pd
import numpy as np
import time
import json

In [26]:
image_path = 'insta_test/test3.png'
img = image.load_img(image_path, target_size=(150, 150))
x = image.img_to_array(img)

In [29]:
x = x.reshape(150,150,3)

In [32]:
import keras
keras.__version__

'2.1.6'

In [33]:
import tensorflow

In [34]:
tensorflow.__version__

'1.10.0'

In [23]:
def solve(image_path, topk = 10):
    time_start = time.clock()
    #load model
    json_file = open('model_image.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    #load weights
    loaded_model.load_weights("image_model_weights_2.h5")
    #load labels
    df_label = pd.read_csv('labels.csv')
    labels = df_label['class'].tolist()
    #load and preprocess image
    img = image.load_img(image_path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = x/255
    preds = loaded_model.predict(x,)
    test_list = preds.tolist()
    probs = []
    for i in test_list:
        for j in i:
            probs.append(j)
    test_df = pd.DataFrame({'label': labels, 'probs': probs})
    test_df = test_df.sort_values('probs', ascending = False)[:topk]
    labels_use = test_df['label'].tolist()
    probs_use = test_df['probs'].tolist()
    dict_json = dict()
    list_probs = []
    for i in range(len(labels_use)):
        dict_test = dict()
        dict_test['name'] = labels_use[i]
        dict_test['probs'] = probs_use[i]
        list_probs.append(dict_test)
    dict_json['data'] = list_probs
    dict_json['time'] = time.clock()-time_start
    dict_json = json.dumps(dict_json)
    return dict_json

In [24]:
solve(image_path)

'{"data": [{"probs": 0.7564043998718262, "name": "TRACIE BAG"}, {"probs": 0.052824195474386215, "name": "JAM 8471"}, {"probs": 0.04148963838815689, "name": "LOLI BAG"}, {"probs": 0.029552597552537918, "name": "JAM 8069"}, {"probs": 0.027224337682127953, "name": "KACAMATA 9127"}, {"probs": 0.020686805248260498, "name": "LILBOW BAG"}, {"probs": 0.02042664960026741, "name": "JAM 8191"}, {"probs": 0.009263462387025356, "name": "MORA WALLET"}, {"probs": 0.006083757616579533, "name": "PAULA BAG"}, {"probs": 0.004621386528015137, "name": "JAM MALE 8036"}], "time": 3.3894916906865546}'

In [14]:
labels_use = df['label'].tolist()
probs_use = df['probs'].tolist()

In [17]:
dict_json = dict()
list_probs = []
for i in range(len(labels_use)):
    dict_test = dict()
    dict_test['name'] = labels_use[i]
    dict_test['probs'] = probs_use[i]
    list_probs.append(dict_test)

In [18]:
dict_json['data'] = list_probs

[{'name': 'TRACIE BAG', 'probs': 0.7564043998718262},
 {'name': 'JAM 8471', 'probs': 0.052824195474386215},
 {'name': 'LOLI BAG', 'probs': 0.04148963838815689},
 {'name': 'JAM 8069', 'probs': 0.029552597552537918},
 {'name': 'KACAMATA 9127', 'probs': 0.027224337682127953},
 {'name': 'LILBOW BAG', 'probs': 0.020686805248260498},
 {'name': 'JAM 8191', 'probs': 0.02042664960026741},
 {'name': 'MORA WALLET', 'probs': 0.009263462387025356},
 {'name': 'PAULA BAG', 'probs': 0.006083757616579533},
 {'name': 'JAM MALE 8036', 'probs': 0.004621386528015137}]

In [ ]:
img = image.load_img(image_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x = x/255

In [ ]:
preds = loaded_model.predict(x,)
pred_classes = np.argmax(preds)
print(preds)
print(pred_classes)

In [ ]:
test_list = preds.tolist()

In [ ]:
probs = []
for i in test_list:
    for j in i:
        probs.append(j)

In [ ]:
test_labels = list(labels.values())

In [ ]:
test_df = pd.DataFrame({'label': test_labels, 'probs': probs})

In [ ]:
test_df.sort_values('probs', ascending = False)

In [ ]:
test_df.sort_values('probs', ascending = False)

## Test 1: Adam
Epoch = 10, loss = categorical_crossentropy, metrics = accuracy  
loss: 0.9872 - acc: 0.7294 - val_loss: 3.0550 - val_acc: 0.406

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)